In [28]:
import os
import datetime

import git
import pandas as pd
from steam.webapi import WebAPI

API_KEY = os.environ['API_KEY']

api = WebAPI(API_KEY, raw=False, format='json', https=True, http_timeout=10)



In [68]:
# Create a DataFrame from the extracted data
aj_data = pd.DataFrame([
    {
        'name':'AJ',
        'date': datetime.date.today(),
        'game': game['name'],
        'playtime_forever': game['playtime_forever']/60
    }
    for game in api.call('IPlayerService.GetOwnedGames', include_appinfo=True, appids_filter=None, include_extended_appinfo=False, include_free_sub=False, include_played_free_games=True, steamid=76561198037729598, language='en')['response']['games']
])

# Create a DataFrame from the extracted data
ashton_data = pd.DataFrame([
    {
        'name':'Ashton',
        'date': datetime.date.today(),
        'game': game['name'],
        'playtime_forever': game['playtime_forever']/60
    }
    for game in api.call('IPlayerService.GetOwnedGames', include_appinfo=True, appids_filter=None, include_extended_appinfo=False, include_free_sub=False, include_played_free_games=True, steamid=76561198102561840, language='en')['response']['games']
])

# Create a DataFrame from the extracted data
dickson_data = pd.DataFrame([
    {
        'name':'Dickson',
        'date': datetime.date.today(),
        'game': game['name'],
        'playtime_forever': game['playtime_forever']/60
    }
    for game in api.call('IPlayerService.GetOwnedGames', include_appinfo=True, appids_filter=None, include_extended_appinfo=False, include_free_sub=False, include_played_free_games=True, steamid=76561198044737410, language='en')['response']['games']

])


In [69]:
aj_data = aj_data[aj_data['playtime_forever']>0]
ashton_data = ashton_data[ashton_data['playtime_forever']>0]
dickson_data = dickson_data[dickson_data['playtime_forever']>0]

playtime_by_date_game = pd.concat([aj_data, ashton_data, dickson_data], axis=0)

In [70]:
playtime_by_date_game

,name,date,game,playtime_forever
0,AJ,2023-08-25,Half-Life,0.116667
1,AJ,2023-08-25,Half-Life 2,0.250000
2,AJ,2023-08-25,Half-Life 2: Lost Coast,0.400000
4,AJ,2023-08-25,Half-Life 2: Deathmatch,0.133333
5,AJ,2023-08-25,Portal,3.300000
...,...,...,...,...
56,Dickson,2023-08-25,Halo Infinite,1.683333
57,Dickson,2023-08-25,MultiVersus,17.550000
58,Dickson,2023-08-25,Call of Duty®,1153.650000
59,Dickson,2023-08-25,Hogwarts Legacy,21.716667


In [73]:
playtime_total = playtime_by_date_game.groupby(['name','date']).sum().drop(columns='game').reset_index()